In [1]:
import pandas as pd
student_data = pd.read_csv('student_data.csv')


C:\Users\tkoehler\AppData\Local\Temp\1\ipykernel_11244\906454212.py:2: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  student_data = pd.read_csv('student_data.csv')


In [2]:
student_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8824678 entries, 0 to 8824677
Data columns (total 14 columns):
 #   Column        Dtype  
---  ------        -----  
 0   sessionId     int64  
 1   timestamp     int64  
 2   x             float64
 3   y             float64
 4   z             float64
 5   ppgValue      float64
 6   hr            float64
 7   hrIbi         float64
 8   hrStatus      float64
 9   ibiStatus     float64
 10  notification  float64
 11  engagement    float64
 12  affect        object 
 13  context       object 
dtypes: float64(10), int64(2), object(2)
memory usage: 942.6+ MB


In [3]:
affect_labels = student_data['affect'].unique()
context_labels = student_data['context'].unique()


In [4]:
nan_percentage = student_data.isnull().mean() * 100


In [5]:
filtered_data_affect = student_data[student_data['affect'].notna()]
filtered_data_context = student_data[student_data['context'].notna()]


In [6]:
import pandas as pd


# First, convert timestamp to seconds for easier computation
student_data['timestamp_seconds'] = student_data['timestamp'] / 1000
print("timestamp conversion to seconds done")
# Function to label surrounding 5 seconds
def label_surrounding_rows(df, label_column):
    # Identify rows where label_column is not NaN
    labeled_indices = df.dropna(subset=[label_column]).index
    
    # For each labeled row, label the surrounding 5 seconds
    for index in labeled_indices:
        # Get the timestamp of the current labeled row
        current_timestamp = df.loc[index, 'timestamp_seconds']
        
        # Define the 5-second window before and after
        window_start = current_timestamp - 5
        window_end = current_timestamp + 5
        
        # Find indices within this window
        window_indices = df[(df['timestamp_seconds'] >= window_start) & (df['timestamp_seconds'] <= window_end)].index
        
        # Apply the label to rows within the window
        df.loc[window_indices, label_column] = df.loc[index, label_column]

    return df

# Apply the function for both 'affect' and 'context' labels
student_data = label_surrounding_rows(student_data, 'affect')
student_data = label_surrounding_rows(student_data, 'context')


timestamp conversion to seconds done


In [7]:
import pandas as pd

# Columns to duplicate values in until a new non-NaN value appears
columns_to_fill = ['hr', 'hrIbi', 'hrStatus', 'ibiStatus']

# Apply forward fill for the specified columns
student_data[columns_to_fill] = student_data[columns_to_fill].fillna(method='ffill')


In [8]:
import pandas as pd


# Drop rows where both 'context' and 'affect' are NaN
cleaned_student_data = student_data.dropna(subset=['context', 'affect'], how='all')

# 'cleaned_student_data' now contains the data without rows where both 'context' and 'affect' were NaN



In [9]:
import pandas as pd
import numpy as np

# Assuming 'student_data' is your DataFrame
# And it contains 'sessionId' as the subject ID column
# Replace 'feature1', 'feature2', etc., with your actual feature column names (e.g., 'x', 'y', 'z', 'hr')

features = ['x', 'y', 'z', 'ppgValue', 'hr', 'hrIbi']  # Add all feature columns you want to normalize

# Function to normalize features within each subject
def normalize_within_subject(df, subject_column, features):
    # Copy the dataframe to avoid modifying the original data
    normalized_df = df.copy()
    
    # Normalize each feature within each subject
    for feature in features:
        print(feature)
        # Apply z-score normalization within each subject
        df_grouped = df.groupby(subject_column)
        normalized_df[feature] = df_grouped[feature].transform(lambda x: (x - x.mean()) / x.std())
    
    # Replace NaN values that result from dividing by std = 0 with 0 (if a feature has constant value)
    normalized_df[features] = normalized_df[features].fillna(0)
    
    return normalized_df

# Apply the normalization function to your data
normalized_student_data = normalize_within_subject(cleaned_student_data, 'sessionId', features)


x
y
z
ppgValue
hr
hrIbi


In [10]:
import pandas as pd

# Assuming 'student_data' is your DataFrame and it's already loaded

# Remove rows where 'x', 'y', and 'z' are all zero
normalized_student_data = normalized_student_data[~((normalized_student_data['x'] == 0) & (normalized_student_data['y'] == 0) & (normalized_student_data['z'] == 0))]


In [11]:
# Assuming 'normalized_student_data' is your DataFrame
normalized_student_data.to_csv('preprocessed_student_data.csv', index=False)
